In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from metal.mmtl.glue_tasks import create_tasks

task_names = [
#     "COLA", 
#     "SST2", 
#     "MNLI",
#     "RTE",
#     "WNLI",
#     "QQP",
    "MRPC",
#     "STSB",
#     "QNLI"
]
tasks = create_tasks(task_names,
                     max_datapoints=200,
                     max_len=200,
                     dl_kwargs={
                         "batch_size": 4
                     }
                    )

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Using random seed: 704119
Loading MRPC Dataset


In [3]:
import copy
import numpy as np
from torch.utils.data import SequentialSampler

def mark_thirds(x):
    tokens = x
    count = len(tokens)
    Y_thirds = np.ceil(np.array([idx * 3/count for idx in range(1, count + 1)])).astype(np.int64)
    return Y_thirds

def overwrite_labels(dataset, label_generator):
    """Creates a copy of dataset where the labels are overwritten with a new labelset
    
    Args:
        dataset: a dataset consisting of (X, Ys)
        label_generator: a function which maps from a single instance x to its new label
    Returns:
        dataset_new: a new dataset where the labels have been overwritten by label_generator
    """
    dataset_new = copy.deepcopy(dataset)
    X = dataset_new.tokens
    Y_new = []
    for x in X:
        Y_new.append(label_generator(x))
    dataset_new.labels = [Y_new]
    return dataset_new

def append_labels(dataset, label_generator):
    X = dataset.tokens
    Y_new = []
    for x in X:
        Y_new.append(label_generator(x))
    dataset_new.labels.append(Y_new)
    return dataset_new    

In [4]:
# print(tasks[0])
# for batch in tasks[0].data_loaders["train"]:
#     X, Y = batch
#     print(Y)
#     break

In [5]:
# data_loader_new = overwrite_labels(tasks[0].data_loaders["train"], mark_thirds)
# for batch in data_loader_new:
#     X, Y = batch
#     print(Y)
#     break

### Create a new task with only token labels

In [7]:
from metal.mmtl.task import TokenClassificationTask
from metal.mmtl.modules import BertTokenClassificationHead

temp_task = copy.deepcopy(tasks[0])
bert_module = temp_task.input_module
data_loaders = {}
for split, loader in temp_task.data_loaders.items():
    loader.dataset = overwrite_labels(loader.dataset, mark_thirds)
    data_loaders[split] = loader

token_head = BertTokenClassificationHead(768, 3)
thirds_task = TokenClassificationTask(
        name="Thirds",
        data_loaders=data_loaders,
        input_module=bert_module,
        head_module=token_head,
)

tasks.append(thirds_task)
# tasks = [thirds_task]
print(tasks)

[TokenClassificationTask(name=Thirds, task_names=Thirds)]


###  Supplement an existing task with additional token labels

In [8]:
# for split, loader in tasks[0].data_loaders.items():
#     loader.dataset = append_labels(loader.dataset, mark_thirds)

# print(tasks)

In [9]:
# import copy
# import numpy as np

# for split, dataloader in tasks[0].data_loaders.items():

#     # To add additional label set (garbage filler right now), uncomment:
#     sentence_labels = dataloader.dataset.labels[0]

#     # Add sentence task
#     # faux_labels = copy.deepcopy(sentence_labels)
#     # dataloader.dataset.labels = [sentence_labels, faux_labels]
#     # tasks[0].task_names = [t.name for t in tasks]

#     # Add token task
#     token_labels = make_labels_marking_thirds(dataloader)
#     dataloader.dataset.labels = [token_labels]

#     dataloader.dataset.labels

In [10]:
# for batch in tasks[0].data_loaders["train"]:
#     X, Ys = batch
#     tokens, segments, masks = X
#     batch_size, seq_len = tokens.shape
#     print(tokens)
#     print(tokens.shape)
#     print(segments)
#     print(masks)
# #     active_lens = masks.sum(dim=1)
# #     print(active_lens)
#     print(Ys)
#     print(Ys[0].shape)
#     break

In [11]:
import os
print('Cuda devices: ', os.environ.get('CUDA_VISIBLE_DEVICES', "[Not Set]"))

Cuda devices:  [Not Set]


In [12]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, verbose=False)

/dfs/scratch0/bradenjh/metal/metal/mmtl/metal_model.py:75: UserWarning: nn.DataParallel has been removed!
  warnings.warn("nn.DataParallel has been removed!")


In [13]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

109484547


In [14]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer()

In [15]:
trainer.train_model(
    model,
    tasks,
    checkpoint_metric="model/train/loss",
    n_epochs=3,
    lr=1e-5,
    log_every=1,
    score_every=1,
    progress_bar=True
)

Beginning train loop.
Expecting a total of approximately 200 examples and 50 batches per epoch from 1 tasks.


[1.0 epo]: TRAIN:[Thirds/loss=9.01e-01, model/loss=9.01e-01, model/lr=1.00e-05] VALID:[Thirds/token_acc=7.75e-01, model/glue=nan]



/dfs/scratch0/bradenjh/metal/metal/mmtl/trainer.py:445: UserWarning: You requested glue score but have fewer than 9 tasks. Be aware.
  warnings.warn(msg)
/dfs/scratch0/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/dfs/scratch0/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[2.0 epo]: TRAIN:[Thirds/loss=3.43e-01, model/loss=3.43e-01, model/lr=1.00e-05] VALID:[Thirds/token_acc=9.26e-01, model/glue=nan]



[3.0 epo]: TRAIN:[Thirds/loss=1.51e-01, model/loss=1.51e-01, model/lr=1.00e-05] VALID:[Thirds/token_acc=9.44e-01, model/glue=nan]

Finished training
{'Thirds/test/token_acc': 0.9441104744879669,
 'Thirds/train/token_acc': 0.9704131616587685,
 'Thirds/valid/token_acc': 0.9441104744879669,
 'model/None/glue': nan}
Cleaning checkpoints


trainer.train_model(
    model,
    tasks,
    checkpoint_metric="model/train/loss",
    n_epochs=1,
    progress_bar=True
)

In [16]:
# import torch
# from metal.utils import padded_tensor

# a_list = [(1,2,3), (1,2,3,4)]
# a, _ = padded_tensor(a_list, dtype=torch.float)
# print(a)
# a = torch.tensor([1,2,3], dtype=torch.long)
# print(isinstance(a, torch.Tensor))
# print(type(a))
# print(a.type())

In [17]:
print("WE WIN!")

WE WIN!
